In [1]:
from functools import reduce
import os
from operator import or_, and_

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dateutil.parser import parse
import django
from django.db.models import Q

from jupyter_plotly_dash import JupyterDash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
# from plotter.views import *
# from plotter.forms import *
from utils import rows, columns

In [2]:
def main_graph():
    return dcc.Graph(
            id = 'main-graph',
            figure = go.Figure(),
            style={'height':'100vh'}
        )

In [16]:
def qlist(queryset, attribute):
    return list(queryset.values_list(attribute, flat=True))

def qset_axes(queryset, x_val, y_val):
    """
    make a pair of flat lists from a queryset based on passed fields.
    for stuff like plots
    this can be made much fancier
    """
    return [qlist(queryset, x_val), qlist(queryset, y_val)]

In [70]:
def scatter(queryset, x_val, y_val):
    fig = go.Figure()
    axes = qset_axes(queryset, x_val, y_val)
    fig.add_trace(go.Scattergl(
        x = axes[0],
        y = axes[1],
        # change this to be hella popup text
        text = None,
        marker = {
            'color':'blue'
        },
        ))
    return fig

In [5]:
def particular_fields_search(model, search_dict, searchable_fields):
    """
    'search specific defined fields' function.
    works only on strings atm!
    other things for numeric / date / etc. fields preferably
    """
    form_results = model.objects.all()
    for field in searchable_fields:
        value = search_dict.get(field)
        if value:
            # allow exact phrase searches
            query = field + "__iexact"
            if form_results.filter(**{query: value}):
                form_results = form_results.filter(**{query: value})
            # otherwise treat multiple words as an 'or' search
            else:
                query = field + "__icontains"
                filters = [
                    form_results.filter(**{query: word}) for word in value.split(" ")
                ]
                form_results = reduce(or_, filters)
    return form_results

In [67]:
def filter_drop(model, element_id):
    return dcc.Dropdown(
        id=element_id,
        options=[{'label': filt, 'value': filt}
                 for filt in model.filters],
        style = {'width':'10rem', 'display':'inline-block'}
    )

In [ ]:
def field_drop(fields, element_id):
    return dcc.Dropdown(
        id=element_id,
        options = [{'label': field, 'value': field}
                 for field in fields]
    )

In [71]:
def model_options_drop(model, field):
    return dcc.Dropdown(
        id=element_id,
        options=[{'label': item, 'value': item}
                 for filt in qlist(model,field)]
    )

In [68]:
#app = JupyterDash('TestGraph')
app = dash.Dash()
fig = main_graph()
#dataset_function = make_mspec_axes
graph_function = scatter
queryset = MSpec.objects.all()


# it's a big open UX question how to gracefully mutate filters between instruments.
# it might be that flexible band values are better than explicit filter
# selection.
app.layout = html.Div(children = [
    filter_drop(MSpec,'filter-1'),
    filter_drop(MSpec,'filter-2'),
    filter_drop(MSpec,'filter-3'),
    dcc.RadioItems(
        id='calc-type',
        options=[{'label': calc, 'value': calc}
                 for calc in ['L2', 'L3', 'L4']],
        value='L2'
    ),
    
    html.Button(id='submit_search', n_clicks=0, children='Submit'),
    html.Div(children = [fig],
    )])


@app.callback(
    dash.dependencies.Output('main-graph', 'figure'),
    [dash.dependencies.Input('filter-1', 'value'),
     dash.dependencies.Input('filter-2', 'value')
    ])
def update_graph_values(filt, filt2):
    # move axes construction to a different function
    return graph_function(queryset, filt+'_mean', filt2+'_mean')

@app.callback(Output('output-state', 'children'),
              [Input('submit-button-state', 'n_clicks')],
              [State('input-1-state', 'value'),
               State('input-2-state', 'value')])
def search(field, query)

In [69]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gun

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jun/2020 04:05:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 04:05:57] "GET /_dash-component-suites/dash_renderer/react@16.v1_5_0m1592852348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 04:05:57] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_5_0m1592852348.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 04:05:57] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_5_0m1592852348.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 04:05:57] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_5_0m1592852348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 04:05:57] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_10_1m1592852348.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 04:05:57] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_3m1592852348.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 04

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/michael/.local/lib/python3.8/site-packages/dash/dash.py",

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/michael/.local/lib/python3.8/site-packages/dash/dash.py",

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/michael/.local/lib/python3.8/site-packages/dash/dash.py",

127.0.0.1 - - [26/Jun/2020 04:05:57] "POST /_dash-update-component HTTP/1.1" 500 -


In [ ]:
# ok so something that wraps querysets for plotly?
# we want both the requested attributes and at least the id,
# name, etc.
# so that people can query more information about chosen spectra

In [15]:
qlist(MSpec.objects.all(), 'L2_mean')

<QuerySet [0.0237608, 0.0280853, 0.0337442, 0.0383073, 0.0387425, 0.0411567, 0.0412882, 0.0419614, 0.0423339, 0.042713, 0.0441043, 0.04674980000000001, 0.047032300000000006, 0.0475577, 0.0480634, 0.0484816, 0.050754, 0.0526092, 0.053677300000000004, 0.0546345, '...(remaining elements truncated)...']>